^fill^README.md^here^

- - -

^fill^theory.md^here^

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import random
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'dynamic_relax'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_mpi'
mpi_command = 'mpiexec -localonly 4'
lib_directory = '../../../library'

#### 2.2 Specify the prototype system

- __system__ is an atomman.System to perform the scan on (required). 

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [4]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.5, 3.5, 3.5]
sizemults = [10,10,10]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 'crystal_prototype', prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

# Generate system by supersizing ucell
system = am.supersize(ucell, *sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in ucell = 4
# of atoms in system = 4000


#### 2.3 Specify the potenital and elemental symbols

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use (required).

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system (required).

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).

In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 'potential_LAMMPS', potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
potential = lmp.Potential(potential_path, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.4 Specify calculation-specific run parameters

- __integrator__ specifies the integrator style to use. Default value is 'nph+l' for temperature = 0, and 'npt' otherwise.

- __pressure_xx__ gives the xx component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_yy__ gives the yy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_zz__ gives the zz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __temperature__ gives the temperature to equilibriate the system to (nvt, npt, nve+l, and nph+l styles).

- __runsteps__ is the total number of integration timesteps to perform. Default value is 220000.
  
- __thermosteps__ specifies to output thermo values every this many timesteps. Default value is 100.
    
- __dumpsteps__ specifies to output dump files every this many timesteps. Default value is runsteps (only first and last steps are outputted as dump files).
    
- __equilsteps__ is the number of timesteps to equilibriate the system for. Only thermo values associated with timesteps greater than equilsteps will be included in the mean and standard deviation calculations. Default value is 20000. 

- __randomseed__ specifies a random number seed used to generate the initial atomic velocities and the Langevin thermostat fluctuations. Default value generates a new random integer every time.

In [6]:
integrator = 'npt'
pressure_xx = uc.set_in_units(0.0, 'GPa')
pressure_yy = uc.set_in_units(0.0, 'GPa')
pressure_zz = uc.set_in_units(0.0, 'GPa')
temperature = 300.0
runsteps = 220000
thermosteps = 100
dumpsteps = runsteps
equilsteps = 20000
randomseed = None

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 full_relax.template

In [7]:
with open('full_relax.template', 'w') as f:
    f.write("""^fill^full_relax.template^here^""")

#### 3.2 integrator_info()

In [8]:
^fill^calc_dynamic_relax.py(integrator_info)^here^

#### 3.3 full_relax()

In [9]:
^fill^calc_dynamic_relax.py(full_relax)^here^

### 4. Run calculation function(s)

In [10]:
results_dict = full_relax(lammps_command, 
                          system, 
                          potential,
                          symbols,
                          mpi_command = mpi_command,
                          ucell=ucell,
                          p_xx=pressure_xx,
                          p_yy=pressure_yy,
                          p_zz=pressure_zz,
                          temperature=temperature,
                          integrator=integrator,
                          runsteps=runsteps,
                          thermosteps=thermosteps,
                          dumpsteps=dumpsteps,
                          equilsteps=equilsteps,
                          randomseed=randomseed)

In [11]:
results_dict.keys()

['E_coh_std',
 'stress',
 'gamma_lat',
 'temp',
 'stress_std',
 'a_lat',
 'beta_lat',
 'b_lat',
 'E_coh',
 'c_lat',
 'alpha_lat',
 'c_lat_std',
 'temp_std',
 'nsamples',
 'a_lat_std',
 'system_relaxed',
 'b_lat_std']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __pressure_unit__ is the unit of pressure to display Cij and relaxed stress state in.

In [12]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2 Display number of samples and number of atoms

In [13]:
print('# of samples = ', results_dict['nsamples'])
print('# of atoms =   ', results_dict['system_relaxed'].natoms)

# of samples =  2001
# of atoms =    4000


#### 5.3 Print Ecoh and lattice constants of relaxed ucell

In [14]:
print('Ecoh =      %9.6f' % uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('std(Ecoh) = %9.6f' % uc.get_in_units(results_dict['E_coh_std'], energy_unit), energy_unit)
print('a =         %9.6f' % uc.get_in_units(results_dict['a_lat'], length_unit), length_unit)
print('std(a) =    %9.6f' % uc.get_in_units(results_dict['a_lat_std'], energy_unit), length_unit)
print('b =         %9.6f' % uc.get_in_units(results_dict['b_lat'], length_unit), length_unit)
print('std(b) =    %9.6f' % uc.get_in_units(results_dict['b_lat_std'], energy_unit), length_unit)
print('c =         %9.6f' % uc.get_in_units(results_dict['c_lat'], length_unit), length_unit)
print('std(c) =    %9.6f' % uc.get_in_units(results_dict['c_lat_std'], energy_unit), length_unit)
print('alpha =     %9.6f' % results_dict['alpha_lat'])
print('beta =      %9.6f' % results_dict['beta_lat'])
print('gamma =     %9.6f' % results_dict['gamma_lat'])

Ecoh =      -4.413040 eV
std(Ecoh) =  0.000484 eV
a =          3.532964 angstrom
std(a) =     0.001598 angstrom
b =          3.533034 angstrom
std(b) =     0.001686 angstrom
c =          3.533102 angstrom
std(c) =     0.001813 angstrom
alpha =     90.000000
beta =      90.000000
gamma =     90.000000


#### 5.4 Check stress state of relaxed system

In [15]:
print('stress ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))
print()
print('std(stress) ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress_std'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))

stress (GPa) =
[   0.0007    0.0015    0.0006]
[   0.0015    0.0011   -0.0004]
[   0.0006   -0.0004    0.0007]

std(stress) (GPa) =
[   0.0879    0.0257    0.0245]
[   0.0257    0.0915    0.0252]
[   0.0245    0.0252    0.0932]


#### 5.5 Check temperature state of relaxed system

In [16]:
print('T =      %9.6f K' % results_dict['temp'])
print('std(T) = %9.6f K' % results_dict['temp_std'])

T =      300.176572 K
std(T) =  3.826057 K
